In [220]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchinfo import summary
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import onnx

In [221]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.28604,), (0.32025,))
])

In [222]:
training_data = datasets.FashionMNIST(
    root="data",  # Directory to save the data in
    train=True,  # Specifies training dataset
    download=True,  # Downloads the dataset if not already present
    transform=transform  # Applies the defined transform
)

In [223]:
# 3. Download and load the test data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,  # Specifies test dataset
    download=True,
    transform=transform
)

# 4. Create DataLoaders for batching and shuffling
BATCH_SIZE = 64

In [224]:
train_loader = DataLoader(
    training_data,
    batch_size=BATCH_SIZE,
    shuffle=True  # Shuffles the data every epoch for better training
)

test_loader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False  # Shuffling not necessary for testing
)

In [225]:
# 5. Iterate through a DataLoader (optional, for verification)
print(f"Number of batches in training loader: {len(train_loader)}")
first_batch_images, first_batch_labels = next(iter(train_loader))
print(f"First batch image shape: {first_batch_images.shape}")
print(f"First batch label shape: {first_batch_labels.shape}")

Number of batches in training loader: 938
First batch image shape: torch.Size([64, 1, 28, 28])
First batch label shape: torch.Size([64])


In [226]:
first_batch_labels

tensor([6, 8, 1, 0, 8, 9, 5, 1, 7, 3, 8, 0, 1, 8, 3, 8, 2, 1, 1, 1, 2, 3, 7, 7,
        6, 3, 1, 7, 7, 3, 0, 3, 7, 6, 4, 6, 2, 3, 3, 1, 1, 8, 7, 9, 2, 9, 4, 4,
        9, 1, 6, 1, 5, 1, 4, 2, 3, 1, 6, 8, 7, 2, 3, 5])

In [227]:
# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [228]:
# 2. Define the Neural Network Modelclass SimpleNN(nn.Module):
class FashionNN(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fc1 = nn.Linear(28 * 28, 20)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [229]:
model = FashionNN().to(device)

In [230]:
# 3. Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [231]:
writer = SummaryWriter('runs/init')
running_loss = 0
# 4. Training Loop
NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backward pass (calculate gradients)
        optimizer.step()  # Update weights

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch + 1}/{NUM_EPOCHS} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    avg_train_loss = running_loss / len(train_loader)
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)

writer.close()

Train Epoch: 1/5 [0/60000 (0%)]	Loss: 2.407512
Train Epoch: 1/5 [6400/60000 (11%)]	Loss: 0.903699
Train Epoch: 1/5 [12800/60000 (21%)]	Loss: 0.516983
Train Epoch: 1/5 [19200/60000 (32%)]	Loss: 0.731142
Train Epoch: 1/5 [25600/60000 (43%)]	Loss: 0.401668
Train Epoch: 1/5 [32000/60000 (53%)]	Loss: 0.404762
Train Epoch: 1/5 [38400/60000 (64%)]	Loss: 0.371117
Train Epoch: 1/5 [44800/60000 (75%)]	Loss: 0.459133
Train Epoch: 1/5 [51200/60000 (85%)]	Loss: 0.285300
Train Epoch: 1/5 [57600/60000 (96%)]	Loss: 0.267680
Train Epoch: 2/5 [0/60000 (0%)]	Loss: 0.576227
Train Epoch: 2/5 [6400/60000 (11%)]	Loss: 0.380893
Train Epoch: 2/5 [12800/60000 (21%)]	Loss: 0.215644
Train Epoch: 2/5 [19200/60000 (32%)]	Loss: 0.766666
Train Epoch: 2/5 [25600/60000 (43%)]	Loss: 0.418699
Train Epoch: 2/5 [32000/60000 (53%)]	Loss: 0.352307
Train Epoch: 2/5 [38400/60000 (64%)]	Loss: 0.320565
Train Epoch: 2/5 [44800/60000 (75%)]	Loss: 0.358250
Train Epoch: 2/5 [51200/60000 (85%)]	Loss: 0.470961
Train Epoch: 2/5 [57600/

In [208]:
# 5. Testing the Model
test_loss = 0
correct = 0
model.eval()

FashionNN(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [209]:
summary(model)

Layer (type:depth-idx)                   Param #
FashionNN                                --
├─Linear: 1-1                            7,850
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0

In [210]:
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

In [211]:
test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
      f'({accuracy:.2f}%)\n')

# 6. Save the model (optional)
torch.save(model.state_dict(), "fashion_nn.pth")


Test set: Average loss: 0.4635, Accuracy: 8376/10000 (83.76%)

